# وكلاء AutoGen في الإنتاج: المراقبة والتقييم

في هذا الدليل التعليمي، سنتعلم كيفية **مراقبة الخطوات الداخلية (التتبع) لوكلاء [Autogen](https://github.com/microsoft/autogen)** و**تقييم أدائهم** باستخدام [Langfuse](https://langfuse.com).

يغطي هذا الدليل مقاييس التقييم **عبر الإنترنت** و**غير المتصلة بالإنترنت** التي تستخدمها الفرق لإدخال الوكلاء إلى الإنتاج بسرعة وموثوقية.

**لماذا يعتبر تقييم وكلاء الذكاء الاصطناعي مهمًا:**
- استكشاف الأخطاء وإصلاحها عند فشل المهام أو إنتاج نتائج دون المستوى المطلوب
- مراقبة التكاليف والأداء في الوقت الفعلي
- تحسين الموثوقية والسلامة من خلال التغذية الراجعة المستمرة


## الخطوة 1: إعداد متغيرات البيئة

احصل على مفاتيح Langfuse API الخاصة بك عن طريق التسجيل في [Langfuse Cloud](https://cloud.langfuse.com/) أو [استضافة Langfuse ذاتيًا](https://langfuse.com/self-hosting).

_**ملاحظة:** يمكن للمستضيفين ذاتيًا استخدام [وحدات Terraform](https://langfuse.com/self-hosting/azure) لنشر Langfuse على Azure. بدلاً من ذلك، يمكنك نشر Langfuse على Kubernetes باستخدام [مخطط Helm](https://langfuse.com/self-hosting/kubernetes-helm)._


In [5]:
import os

# Get keys for your project from the project settings page: https://cloud.langfuse.com
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-..." 
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-..." 
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com" # 🇪🇺 EU region
# os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com" # 🇺🇸 US region

مع تعيين متغيرات البيئة، يمكننا الآن تهيئة عميل Langfuse. تقوم `get_client()` بتهيئة عميل Langfuse باستخدام بيانات الاعتماد المقدمة في متغيرات البيئة.


In [6]:
from langfuse import Langfuse
 
# Filter out Autogen OpenTelemetryspans
langfuse = Langfuse(
    blocked_instrumentation_scopes=["autogen SingleThreadedAgentRuntime"]
)
 
# Verify connection
if langfuse.auth_check():
    print("Langfuse client is authenticated and ready!")
else:
    print("Authentication failed. Please check your credentials and host.")

Langfuse client is authenticated and ready!


## الخطوة 2: تهيئة أدوات OpenLit

الآن، نقوم بتهيئة أدوات [OpenLit](https://github.com/openlit/openlit). يقوم OpenLit تلقائيًا بالتقاط عمليات AutoGen وتصدير نطاقات OpenTelemetry (OTel) إلى Langfuse.


In [7]:
import openlit
 
# Initialize OpenLIT instrumentation. The disable_batch flag is set to true to process traces immediately.
openlit.init(tracer=langfuse._otel_tracer, disable_batch=True, disabled_instrumentors=["mistral"])

## الخطوة 3: تشغيل الوكيل الخاص بك

الآن نقوم بإعداد وكيل متعدد الأدوار لاختبار أدوات القياس الخاصة بنا.


In [2]:
import os

from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_agentchat.base import TaskResult

from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat

In [3]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
        "structured_output": False
    },
)

In [8]:
# 🍴 Agent 1 – proposes ONE healthy meal idea each turn
meal_planner_agent = AssistantAgent(
    "meal_planner_agent",
    model_client=client,
    description="A seasoned meal-planning coach who suggests balanced meals.",
    system_message="""
    You are a Meal-Planning Assistant with a decade of experience helping busy people prepare meals.
    Goal: propose the single best meal (breakfast, lunch, or dinner) given the user's context.
    Each response must contain ONLY one complete meal idea (title + very brief component list) — no extras.
    Keep it concise: skip greetings, chit-chat, and filler.
    """,
)

# 🥗 Agent 2 – checks nutritional quality & variety
nutritionist_agent = AssistantAgent(
    "nutritionist_agent",
    model_client=client,
    description="A registered dietitian ensuring meals meet nutritional standards.",
    system_message="""
    You are a Nutritionist focused on whole-food, macro-balanced eating.
    Evaluate the meal_planner_agent’s recommendation.
    If the meal is nutritionally sound, sufficiently varied, and portion-appropriate, respond with 'APPROVE'.
    Otherwise, give high-level guidance on how to improve it (e.g. 'add a plant-based protein') — do NOT provide a full alternative recipe.
    """,
)

In [9]:
# ✅ Chat stops once the nutritionist says APPROVE
termination = TextMentionTermination("APPROVE")

# 🔄 Alternate turns between the two agents until termination
team = RoundRobinGroupChat(
    [meal_planner_agent, nutritionist_agent],
    termination_condition=termination,
)

# Example kickoff
user_input = "I'm looking for a quick, delicious dinner I can prep after work. I have 30 minutes and minimal clean-up is ideal."

In [ ]:
with langfuse.start_as_current_span(name="create_meal_plan") as span:
    async for message in team.run_stream(task=user_input):
        if isinstance(message, TaskResult):
            print("Stop Reason:", message.stop_reason)
        else:
            print(message)

    span.update_trace(
        input=user_input,
        output=message.stop_reason,
    )

# Flush the trace to Langfuse for short-lived environments such as Jupyter Notebooks
langfuse.flush()

### هيكل التتبع

يسجل Langfuse **تتبعًا** يحتوي على **فترات زمنية**، والتي تمثل كل خطوة من منطق وكيلك. هنا، يحتوي التتبع على تشغيل الوكيل بشكل عام وفترات زمنية فرعية لـ:
- وكيل تخطيط الوجبات
- وكلاء التغذية

يمكنك فحص هذه الفترات لمعرفة بالضبط أين يتم قضاء الوقت، وعدد الرموز المستخدمة، وما إلى ذلك:

![شجرة التتبع في Langfuse](https://langfuse.com/images/cookbook/example-autogen-evaluation/trace-tree.png)

_[رابط إلى التتبع](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/dac2b33e7cd709e685ccf86a137ecc64)_


## التقييم عبر الإنترنت

يشير التقييم عبر الإنترنت إلى تقييم الوكيل في بيئة حقيقية ومباشرة، أي أثناء الاستخدام الفعلي في الإنتاج. يتضمن ذلك مراقبة أداء الوكيل في تفاعلات المستخدمين الحقيقية وتحليل النتائج بشكل مستمر.

### المقاييس الشائعة التي يجب تتبعها في الإنتاج

1. **التكاليف** — تقوم الأدوات بتسجيل استخدام الرموز، ويمكنك تحويل ذلك إلى تكاليف تقريبية من خلال تحديد سعر لكل رمز.
2. **الزمن المستغرق** — راقب الوقت الذي يستغرقه إكمال كل خطوة أو العملية بأكملها.
3. **ملاحظات المستخدمين** — يمكن للمستخدمين تقديم ملاحظات مباشرة (إعجاب/عدم إعجاب) للمساعدة في تحسين أو تصحيح أداء الوكيل.
4. **LLM كحكم** — استخدم نموذج لغة منفصل لتقييم مخرجات الوكيل في الوقت القريب من الحقيقي (مثل التحقق من السمية أو الدقة).

فيما يلي أمثلة لهذه المقاييس.


#### 1. التكاليف

فيما يلي لقطة شاشة توضح استخدام مكالمات `gpt-4o-mini`. هذا مفيد لرؤية الخطوات المكلفة وتحسين وكيلك.

![التكاليف](https://langfuse.com/images/cookbook/example-autogen-evaluation/gpt-4o-costs.png) 

_[رابط التتبع](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/dac2b33e7cd709e685ccf86a137ecc64)_


#### 2. التأخير

يمكننا أيضًا معرفة المدة التي استغرقتها كل خطوة لإكمالها. في المثال أدناه، استغرقت العملية بأكملها حوالي 3 ثوانٍ، ويمكنك تقسيمها حسب الخطوات. يساعدك هذا في تحديد نقاط الاختناق وتحسين أداء الوكيل.

![التأخير](https://langfuse.com/images/cookbook/example-autogen-evaluation/agent-latency.png) 

_[رابط إلى التتبع](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/dac2b33e7cd709e685ccf86a137ecc64?display=timeline)_


#### 3. ملاحظات المستخدم

إذا كان وكيلك مدمجًا في واجهة مستخدم، يمكنك تسجيل ملاحظات المستخدم المباشرة (مثل الإعجاب أو عدم الإعجاب في واجهة دردشة).


In [10]:
from langfuse import get_client
 
langfuse = get_client()
 
# Option 1: Use the yielded span object from the context manager
with langfuse.start_as_current_span(
    name="autogen-request-user-feedback-1") as span:
    
    async for message in team.run_stream(task="Create a meal with potatoes"):
            if isinstance(message, TaskResult):
                print("Stop Reason:", message.stop_reason)
            else:
                print(message)    
 
    # Score using the span object
    span.score_trace(
        name="user-feedback",
        value=1,
        data_type="NUMERIC",
        comment="This was delicious, thank you"
    )
 
# Option 2: Use langfuse.score_current_trace() if still in context
with langfuse.start_as_current_span(name="autogen-request-user-feedback-2") as span:
    # ... Autogen execution ...

    async for message in team.run_stream(task="I am allergic to gluten."):
            if isinstance(message, TaskResult):
                print("Stop Reason:", message.stop_reason)
            else:
                print(message)    
 
    # Score using current context
    langfuse.score_current_trace(
        name="user-feedback",
        value=1,
        data_type="NUMERIC"
    )

id='da068880-22ae-4f01-9f01-2bb231939089' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 20, 43, 732669, tzinfo=datetime.timezone.utc) content='Create a meal with potatoes' type='TextMessage'
id='ad937ce4-3534-493f-824b-ca9c226b5287' source='meal_planner_agent' models_usage=RequestUsage(prompt_tokens=95, completion_tokens=30) metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 20, 45, 186423, tzinfo=datetime.timezone.utc) content='Potato and Spinach Frittata  \n- Eggs  \n- Potatoes  \n- Fresh spinach  \n- Onion  \n- Cheese (optional)  ' type='TextMessage'
id='50fd33c1-057f-49fe-afad-ee86d164296d' source='nutritionist_agent' models_usage=RequestUsage(prompt_tokens=132, completion_tokens=4) metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 20, 45, 581059, tzinfo=datetime.timezone.utc) content='APPROVE' type='TextMessage'
Stop Reason: Text 'APPROVE' mentioned
id='e371de6c-e5fc-42c1-8eda-e5b8cd5accab' source='user' models_usage=None met

In [ ]:
# Option 3: Use create_score() with trace ID (when outside context)
langfuse.create_score(
    trace_id="predefined_trace_id",
    name="user-feedback",
    value=1,
    data_type="NUMERIC",
    comment="This was correct, thank you"
)

يتم جمع ملاحظات المستخدم في Langfuse:

![يتم جمع ملاحظات المستخدم في Langfuse](https://langfuse.com/images/cookbook/example-autogen-evaluation/user-feedback.png)


#### 4. التقييم التلقائي باستخدام LLM-as-a-Judge

LLM-as-a-Judge هو طريقة أخرى لتقييم مخرجات الوكيل الخاص بك تلقائيًا. يمكنك إعداد استدعاء منفصل لنموذج LLM لتقييم صحة المخرجات، أو مدى سميتها، أو أسلوبها، أو أي معايير أخرى تهمك.

**سير العمل**:
1. تقوم بتحديد **قالب تقييم**، على سبيل المثال: "تحقق مما إذا كان النص سامًا."
2. تحدد نموذجًا يُستخدم كنموذج الحكم؛ في هذه الحالة `gpt-4o-mini` الذي يتم استدعاؤه عبر Azure.
3. في كل مرة يقوم وكيلك بإنتاج مخرجات، تقوم بتمرير تلك المخرجات إلى نموذج LLM "الحكم" مع القالب.
4. يستجيب نموذج LLM الحكم بتقييم أو تصنيف تقوم بتسجيله في أداة المراقبة الخاصة بك.

مثال من Langfuse:

![LLM-as-a-Judge Evaluator](https://langfuse.com/images/cookbook/example-autogen-evaluation/evaluator.png)


In [12]:
with langfuse.start_as_current_span(name="autogen-request-user-feedback-2") as span:

    async for message in team.run_stream(task="I am a picky eater and not sure if you find something for me."):
            if isinstance(message, TaskResult):
                print("Stop Reason:", message.stop_reason)
            else:
                print(message) 

    span.update_trace(
        input=user_input,
        output=message.stop_reason,
    )

langfuse.flush()

id='eefc628d-502f-451a-8f70-be486f62f8c5' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 38, 29, 171393, tzinfo=datetime.timezone.utc) content='I am a picky eater and not sure if you find something for me.' type='TextMessage'
id='13b3e14b-bcf7-42a5-80d6-54b0c7be765e' source='meal_planner_agent' models_usage=RequestUsage(prompt_tokens=352, completion_tokens=27) metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 38, 30, 433516, tzinfo=datetime.timezone.utc) content='Chicken Alfredo Pasta  \n- Gluten-free pasta  \n- Grilled chicken breast  \n- Heavy cream  \n- Parmesan cheese  \n- Garlic  ' type='TextMessage'
id='550f2dee-0e08-4bbd-b67f-991b467328f1' source='nutritionist_agent' models_usage=RequestUsage(prompt_tokens=386, completion_tokens=17) metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 38, 31, 505173, tzinfo=datetime.timezone.utc) content='Consider incorporating some vegetables, like spinach or broccoli, to increase the nutrien

يمكنك أن ترى أن إجابة هذا المثال تُعتبر "غير ضارة".

![تقييم درجة LLM-as-a-Judge](https://langfuse.com/images/cookbook/example-autogen-evaluation/llm-as-a-judge-score.png)


#### 5. نظرة عامة على مقاييس المراقبة

يمكن عرض جميع هذه المقاييس معًا في لوحات التحكم. يتيح لك ذلك رؤية أداء وكيلك بسرعة عبر العديد من الجلسات ويساعدك على تتبع مقاييس الجودة على مدار الوقت.

![نظرة عامة على مقاييس المراقبة](https://langfuse.com/images/cookbook/example-autogen-evaluation/dashboard.png)


## التقييم دون اتصال

يُعد التقييم عبر الإنترنت ضروريًا للحصول على ملاحظات فورية، ولكنك تحتاج أيضًا إلى **التقييم دون اتصال**—وهو إجراء فحوصات منهجية قبل أو أثناء التطوير. يساعد ذلك في الحفاظ على الجودة والموثوقية قبل إدخال التغييرات إلى بيئة الإنتاج.


### تقييم مجموعة البيانات

في التقييم غير المتصل بالإنترنت، عادةً ما تقوم بـ:
1. الحصول على مجموعة بيانات مرجعية (تتضمن أزواج من التعليمات والمخرجات المتوقعة)
2. تشغيل وكيلك على تلك المجموعة
3. مقارنة المخرجات بالنتائج المتوقعة أو استخدام آلية تقييم إضافية

فيما يلي، نوضح هذا النهج باستخدام [q&a-dataset](https://huggingface.co/datasets/junzhang1207/search-dataset)، التي تحتوي على أسئلة وإجابات متوقعة.


In [16]:
import pandas as pd
from datasets import load_dataset
 
# Fetch search-dataset from Hugging Face
dataset = load_dataset("junzhang1207/search-dataset", split = "train")
df = pd.DataFrame(dataset)
print("First few rows of search-dataset:")
print(df.head())

/Users/jannik/Documents/GitHub/ai-agents-for-beginners/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


First few rows of search-dataset:
                                     id  \
0  20caf138-0c81-4ef9-be60-fe919e0d68d4   
1  1f37d9fd-1bcc-4f79-b004-bc0e1e944033   
2  76173a7f-d645-4e3e-8e0d-cca139e00ebe   
3  5f5ef4ca-91fe-4610-a8a9-e15b12e3c803   
4  64dbed0d-d91b-4acd-9a9c-0a7aa83115ec   

                                            question  \
0                 steve jobs statue location budapst   
1  Why is the Battle of Stalingrad considered a t...   
2  In what year did 'The Birth of a Nation' surpa...   
3  How many Russian soldiers surrendered to AFU i...   
4   What event led to the creation of Google Images?   

                                     expected_answer       category       area  
0  The Steve Jobs statue is located in Budapest, ...           Arts  Knowledge  
1  The Battle of Stalingrad is considered a turni...   General News       News  
2  This question is based on a false premise. 'Th...  Entertainment       News  
3  About 300 Russian soldiers surrendered to t

بعد ذلك، نقوم بإنشاء كيان مجموعة بيانات في Langfuse لتتبع العمليات. ثم، نضيف كل عنصر من مجموعة البيانات إلى النظام.


In [17]:
from langfuse import Langfuse
langfuse = Langfuse()
 
langfuse_dataset_name = "qa-dataset_autogen-agent"
 
# Create a dataset in Langfuse
langfuse.create_dataset(
    name=langfuse_dataset_name,
    description="q&a dataset uploaded from Hugging Face",
    metadata={
        "date": "2025-03-21",
        "type": "benchmark"
    }
)

Dataset(id='cmcm7524d00kjad07s2cjwqcf', name='qa-dataset_autogen-agent', description='q&a dataset uploaded from Hugging Face', metadata={'date': '2025-03-21', 'type': 'benchmark'}, project_id='cloramnkj0002jz088vzn1ja4', created_at=datetime.datetime(2025, 7, 2, 16, 54, 7, 357000, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2025, 7, 2, 16, 54, 7, 357000, tzinfo=datetime.timezone.utc))

In [18]:
df_25 = df.sample(25) # For this example, we upload only 25 dataset questions

for idx, row in df_25.iterrows():
    langfuse.create_dataset_item(
        dataset_name=langfuse_dataset_name,
        input={"text": row["question"]},
        expected_output={"text": row["expected_answer"]}
    )

![عناصر مجموعة البيانات في Langfuse](https://langfuse.com/images/cookbook/example-autogen-evaluation/example-dataset.png)


#### تشغيل الوكيل على مجموعة البيانات

أولاً، نقوم بتجميع وكيل Autogen بسيط يجيب على الأسئلة باستخدام نماذج Azure OpenAI.


In [8]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_agentchat.messages import TextMessage

In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    max_tokens=5000,
    model_info={
        "json_output": True,
        "function_calling": False,
        "vision": False,
        "family": "unknown",
        "structured_output": True,
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

In [18]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are participant in a quizz show and you are given a question. You need to create a short answer to the question.",
)

ثم، نقوم بتعريف دالة مساعدة `my_agent()`.


In [19]:
async def my_agent(user_query: str):

    with langfuse.start_as_current_span(name="autogen-trace") as span:

        # Execute the agent response
        response = await agent.on_messages(
            [TextMessage(content=user_query, source="user")],
            cancellation_token=CancellationToken(),
        )

        span.update_trace(
            input=user_query,
            output=response.chat_message.content,
        )

    return str(response.chat_message.content)

# Test the function
await my_agent("What is the capital of France?")

'The capital of France is Paris.'

أخيرًا، نقوم بالتكرار على كل عنصر من عناصر مجموعة البيانات، تشغيل الوكيل، وربط التتبع بعنصر مجموعة البيانات. يمكننا أيضًا إرفاق درجة تقييم سريعة إذا لزم الأمر.


In [20]:
dataset_name = "qa-dataset_autogen-agent"
current_run_name = "dev_tasks_run-autogen_gpt-4.1" # Identifies this specific evaluation run
current_run_metadata={"model_provider": "Azure", "model": "gpt-4.1"}
current_run_description="Evaluation run for Autogen model on July 3rd"

dataset = langfuse.get_dataset('qa-dataset_autogen-agent')

for item in dataset.items:
    print(f"Running evaluation for item: {item.id} (Input: {item.input})")
 
    # Use the item.run() context manager
    with item.run(
        run_name=current_run_name,
        run_metadata=current_run_metadata,
        run_description=current_run_description
    ) as root_span: 
        # All subsequent langfuse operations within this block are part of this trace.
        generated_answer = await my_agent(user_query = item.input["text"])
    
    print("Generated Answer: ", generated_answer)
 
print(f"\nFinished processing dataset '{dataset_name}' for run '{current_run_name}'.")

langfuse.flush()

Running evaluation for item: 09810cc4-9992-4712-a3b2-7224da31776a (Input: {'text': 'In Hindu mythology, which deity is the Ganges river dolphin associated with?'})
Generated Answer:  In Hindu mythology, the Ganges river dolphin is associated with the deity Ganga.
Running evaluation for item: bb113f94-7723-47c6-8c34-59d883044514 (Input: {'text': 'What significant discovery did the LHCb collaboration report in 2015?'})
Generated Answer:  In 2015, the LHCb collaboration reported the discovery of pentaquark particles.
Running evaluation for item: 4d8ae54e-ceab-46d0-ad2c-6e8e223589a9 (Input: {'text': 'What is the MÄ\x81ori name for the red-crowned parakeet?'})
Generated Answer:  The Māori name for the red-crowned parakeet is kākāriki.
Running evaluation for item: 21e5a0d5-f619-4a73-868e-9955053b3e72 (Input: {'text': 'Who starred in the 1978 television film adaptation of Les MisÃ©rables?'})
Generated Answer:  Richard Jordan starred as Jean Valjean in the 1978 television film adaptation of Le

يمكنك تكرار هذه العملية باستخدام تكوينات مختلفة للوكيل مثل:
- النماذج (gpt-4o-mini، gpt-4.1، إلخ.)
- التعليمات
- الأدوات (البحث مقابل عدم البحث)
- تعقيد الوكيل (وكيل متعدد مقابل وكيل فردي)

ثم قم بمقارنتها جنبًا إلى جنب في Langfuse. في هذا المثال، قمت بتشغيل الوكيل 3 مرات على 25 سؤالًا من مجموعة البيانات. في كل تشغيل، استخدمت نموذجًا مختلفًا من Azure OpenAI. يمكنك ملاحظة أن عدد الأسئلة التي تمت الإجابة عليها بشكل صحيح يتحسن عند استخدام نموذج أكبر (كما هو متوقع). يتم إنشاء درجة `correct_answer` بواسطة [LLM-as-a-Judge Evaluator](https://langfuse.com/docs/scores/model-based-evals) الذي تم إعداده لتقييم صحة الإجابة بناءً على الإجابة النموذجية المقدمة في مجموعة البيانات.

![نظرة عامة على تشغيل مجموعة البيانات](https://langfuse.com/images/cookbook/example-autogen-evaluation/dataset_runs.png)
![مقارنة تشغيل مجموعة البيانات](https://langfuse.com/images/cookbook/example-autogen-evaluation/dataset-run-comparison.png)



---

**إخلاء المسؤولية**:  
تمت ترجمة هذه الوثيقة باستخدام خدمة الترجمة الآلية [Co-op Translator](https://github.com/Azure/co-op-translator). على الرغم من أننا نسعى لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو معلومات غير دقيقة. يجب اعتبار الوثيقة الأصلية بلغتها الأصلية المصدر الموثوق. للحصول على معلومات حساسة أو هامة، يُوصى بالاستعانة بترجمة بشرية احترافية. نحن غير مسؤولين عن أي سوء فهم أو تفسيرات خاطئة تنشأ عن استخدام هذه الترجمة.
